
# 🌳 Árboles de Decisión — Versión Explicativa (v2)  
**Curso de Análisis de Datos con IA** · **Autores:** Doribel Rodríguez y Antonio Vantaggiato

**Objetivo pedagógico.** Comprender a fondo cómo funcionan los **árboles de decisión**, entrenar y evaluar modelos con un **dataset clásico (Iris)** y con un **caso aplicado educativo** (aprobación de estudiantes), además de explorar hiperparámetros clave para controlar el sobreajuste.



## 🧠 Conceptos esenciales

- **Árbol de decisión (clasificación).** Modelo que divide el espacio de características mediante reglas **if/else** para predecir una clase.  
- **Nodos y hojas.** Cada **nodo** aplica una condición (por ejemplo, `petal_length ≤ 2.4`); cada **hoja** asigna una clase.  
- **Criterios de partición.**  
  - **Gini**: mide impureza de una partición.  
  - **Entropía**: mide desorden; ambas buscan **máxima pureza** de las hojas.  
- **Sobreajuste.** Árboles muy profundos memorizan el conjunto de entrenamiento: baja generalización.  
- **Control de complejidad.** `max_depth`, `min_samples_leaf`, `min_samples_split`, **poda de coste-complejidad**.

> Intuición: un buen árbol encuentra **cortes simples** que separan clases con la **menor impureza** posible.


## 1) Preparación del entorno

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn import datasets

%matplotlib inline



## 2) Dataset clásico: **Iris** (clasificación de flores)

Este dataset contiene 150 flores (3 especies). Usaremos 4 características: largo/ancho de sépalo y pétalo.  
Evaluaremos un árbol básico y analizaremos la matriz de confusión.


In [ ]:

iris = datasets.load_iris()
X = iris.data
y = iris.target
feature_names = iris.feature_names
target_names = iris.target_names

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

arbol = DecisionTreeClassifier(random_state=42, max_depth=None, criterion="gini")
arbol.fit(X_train, y_train)

y_pred = arbol.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Exactitud (accuracy) en test: {acc:.3f}")

cm = confusion_matrix(y_test, y_pred, labels=arbol.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=target_names[arbol.classes_])
disp.plot(cmap="Blues")
plt.title("Matriz de confusión — Iris")
plt.show()

plt.figure(figsize=(12,7))
plot_tree(arbol, feature_names=feature_names, class_names=target_names, filled=True, rounded=True)
plt.title("Árbol de decisión — Iris")
plt.show()



### 2.1) Explorando la **profundidad máxima** (`max_depth`)

La profundidad controla la complejidad del árbol. Probamos varios valores y observamos la exactitud.


In [ ]:

depths = range(1, 11)
scores = []
for d in depths:
    clf = DecisionTreeClassifier(max_depth=d, random_state=42)
    cv = cross_val_score(clf, X, y, cv=5)
    scores.append(cv.mean())

plt.figure(figsize=(7,4))
plt.plot(depths, scores, marker="o")
plt.xlabel("max_depth")
plt.ylabel("Accuracy CV (5-fold)")
plt.title("Elección de profundidad — Validación cruzada")
plt.grid(True)
plt.show()



## 3) Caso aplicado educativo: **Aprobación de estudiantes** (dataset simulado realista)

Variables (simuladas):  
- **asistencia_%**, **horas_estudio**, **participa_clase** (0/1), **tareas_entregadas** (0–10), **promedio_prev** (0–100)  
- **aprobado** (0/1).

Entrenaremos un árbol y veremos **importancias de características**.


In [ ]:

rng = np.random.default_rng(42)
n = 180

asistencia = rng.normal(80, 10, n).clip(40, 100)
horas = rng.normal(6, 2, n).clip(0, 12)
participa = rng.integers(0, 2, n)
tareas = rng.integers(0, 11, n)
prom_prev = rng.normal(75, 12, n).clip(0, 100)

# Regla latente para generar aprobado (probabilística)
logit = -6 + 0.04*asistencia + 0.35*horas + 0.6*participa + 0.18*tareas + 0.03*prom_prev
prob = 1 / (1 + np.exp(-logit))
aprobado = (rng.uniform(0,1,n) < prob).astype(int)

df = pd.DataFrame({
    "asistencia_%": asistencia.round(1),
    "horas_estudio": horas.round(1),
    "participa_clase": participa,
    "tareas_entregadas": tareas,
    "promedio_prev": prom_prev.round(1),
    "aprobado": aprobado
})
df.head()


In [ ]:

X2 = df[["asistencia_%","horas_estudio","participa_clase","tareas_entregadas","promedio_prev"]]
y2 = df["aprobado"]

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.25, random_state=42, stratify=y2)

tree2 = DecisionTreeClassifier(random_state=42, max_depth=5)
tree2.fit(X2_train, y2_train)

y2_pred = tree2.predict(X2_test)
acc2 = accuracy_score(y2_test, y2_pred)
print(f"Exactitud en test (dataset educativo): {acc2:.3f}")

cm2 = confusion_matrix(y2_test, y2_pred, labels=tree2.classes_)
disp2 = ConfusionMatrixDisplay(confusion_matrix=cm2, display_labels=tree2.classes_)
disp2.plot(cmap="Greens")
plt.title("Matriz de confusión — Estudiantes")
plt.show()

# Importancias
importancias = pd.Series(tree2.feature_importances_, index=X2.columns).sort_values(ascending=False)
print("Importancia de características:")
display(importancias)

plt.figure(figsize=(6,4))
importancias.plot(kind="bar")
plt.title("Importancia de características — Árbol (estudiantes)")
plt.ylabel("Importancia")
plt.grid(axis="y")
plt.show()

plt.figure(figsize=(12,7))
plot_tree(tree2, feature_names=X2.columns, class_names=["No","Sí"], filled=True, rounded=True)
plt.title("Árbol de decisión — Aprobación de estudiantes")
plt.show()



## 4) Cierre
- La **profundidad** del árbol y parámetros como `min_samples_leaf` afectan el equilibrio **sesgo–varianza**.  
- En el caso educativo, variables como **tareas** y **horas de estudio** suelen tener alta importancia.  
- Mantén prácticas responsables: valida con datos independientes y revisa sesgos del conjunto.
